# Transformations on a JSON file using Pandas

This is a notebook for the medium article [Transformations on a JSON file using Pandas](https://medium.com/p/eba831181a96)

Please check out article for instructions

License: [MIT](https://opensource.org/licenses/MIT)

In [ ]:
import pandas as pd

## 1. Explode

In [7]:
data = {
    'col_1': [True, False],
    'col_2': [['one', 'two'], ['three']]
}
df = pd.DataFrame(data)
df

,col_1,col_2
0,True,"[one, two]"
1,False,[three]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col_1   2 non-null      bool  
 1   col_2   2 non-null      object
dtypes: bool(1), object(1)
memory usage: 146.0+ bytes


In [9]:
exploded_df = df.explode('col_2')
exploded_df

,col_1,col_2
0,True,one
0,True,two
1,False,three


In [10]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col_1   3 non-null      bool  
 1   col_2   3 non-null      object
dtypes: bool(1), object(1)
memory usage: 51.0+ bytes


## 2. JSON Normalize

In [11]:
data = [
  {
    "id": 1,
    "name": {
      "first": "Coleen",
      "last": "Volk"
    }
  },
  {
    "name": {
      "given": "Mark",
      "family": "Regner"
    }
  },
  {
    "id": 2,
    "name": "Faye Raker"
  },
  
]
df = pd.json_normalize(data)
df

,id,name.first,name.last,name.given,name.family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2 non-null      float64
 1   name.first   1 non-null      object 
 2   name.last    1 non-null      object 
 3   name.given   1 non-null      object 
 4   name.family  1 non-null      object 
 5   name         1 non-null      object 
dtypes: float64(1), object(5)
memory usage: 272.0+ bytes


In [15]:
df = pd.json_normalize(data, sep=' | ')
df

,id,name | first,name | last,name | given,name | family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


In [16]:
df = pd.json_normalize(data, max_level=0)
df

,id,name
0,1.0,"{'first': 'Coleen', 'last': 'Volk'}"
1,NaN,"{'given': 'Mark', 'family': 'Regner'}"
2,2.0,Faye Raker


In [17]:
df = pd.json_normalize(data, max_level=1)
df

,id,name.first,name.last,name.given,name.family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


## 3. Example

In [20]:
data = [
 {
   "name": "mike",
   "age": 19,
   "grades": [
      { 
        "class":'math',
        "grade": 8
      }
   ]
 },
 {
   "name": "john",
   "age": 19,
   "grades": [
      { 
        "class":'math',
        "grade": 9
      },
      { 
        "class":'science',
        "grade": 10
      },
   ]
 }
]

In [22]:
df = pd.DataFrame.from_dict(data)
df

,name,age,grades
0,mike,19,"[{'class': 'math', 'grade': 8}]"
1,john,19,"[{'class': 'math', 'grade': 9}, {'class': 'sci..."


### 3.1 Explode the example

In [31]:
exploded_df = df.explode('grades')
exploded_df

,name,age,grades
0,mike,19,"{'class': 'math', 'grade': 8}"
1,john,19,"{'class': 'math', 'grade': 9}"
1,john,19,"{'class': 'science', 'grade': 10}"


In [25]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    3 non-null      object
 1   age     3 non-null      int64 
 2   grades  3 non-null      object
dtypes: int64(1), object(2)
memory usage: 96.0+ bytes


### 3.2 Normalize the example

In [26]:
normalized_df = pd.json_normalize(exploded_df['grades'])
normalized_df

,class,grade
0,math,8
1,math,9
2,science,10


In [27]:
normalized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   3 non-null      object
 1   grade   3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


### 3.3 Join

In [32]:
exploded_df.drop(['grades'], axis=1, inplace=True)
exploded_df.reset_index(inplace=True)
exploded_df

,index,name,age
0,0,mike,19
1,1,john,19
2,1,john,19


In [33]:
joined_df = exploded_df.join(normalized_df)
joined_df

,index,name,age,class,grade
0,0,mike,19,math,8
1,1,john,19,math,9
2,1,john,19,science,10


In [34]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   3 non-null      int64 
 1   name    3 non-null      object
 2   age     3 non-null      int64 
 3   class   3 non-null      object
 4   grade   3 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 248.0+ bytes


### 3.4 Transformations

In [44]:
threshold = 9

passed_col = joined_df.apply(lambda row: row.grade >= threshold, axis=1)
passed_col

0    False
1     True
2     True
dtype: bool

In [45]:
joined_df['passed'] = passed_col
joined_df

,index,name,age,class,grade,passed
0,0,mike,19,math,8,False
1,1,john,19,math,9,True
2,1,john,19,science,10,True


In [47]:
from datetime import datetime

In [55]:
now = datetime.now()

timestamp = datetime.timestamp(now)

joined_df['processed_ts'] = timestamp
joined_df

,index,name,age,class,grade,passed,processed_ts
0,0,mike,19,math,8,False,1.663841e+09
1,1,john,19,math,9,True,1.663841e+09
2,1,john,19,science,10,True,1.663841e+09


In [56]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         3 non-null      int64  
 1   name          3 non-null      object 
 2   age           3 non-null      int64  
 3   class         3 non-null      object 
 4   grade         3 non-null      int64  
 5   passed        3 non-null      bool   
 6   processed_ts  3 non-null      float64
dtypes: bool(1), float64(1), int64(3), object(2)
memory usage: 275.0+ bytes


### 3.5 Dump the transformed data as CSV

In [59]:
joined_df.to_csv('processed.csv', index=False)